In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
%run train_and_run_experiments_bc.py --dataset sst --data_dir . --output_dir outputs/ --attention tanh

In [ ]:
from Trainers.DatasetBC import *
from ExperimentsBC import *

In [ ]:
from Trainers.DatasetQA import *
from ExperimentsQA import *

In [ ]:
for k in datasets :
    print(k)
    if k == 'sst' :
        dataset = datasets[k]()
        generate_graphs_on_encoders(dataset)
        print('+'*700)

In [ ]:
dataset = datasets_ehr['mortality']()

In [ ]:
from Trainers.PlottingBC import *

In [ ]:
evaluator = run_evaluator_on_latest_model(dataset, config='lstm')
logodds_results = pload(evaluator.model, 'logodds_attention')
emax_jds, emax_adv_attn, emax_ad_y = plot_attn_diff(dataset, dataset.test_data, logodds_results, 
                                                    save_name='logodds_subs', dirname=evaluator.model.dirname)

In [ ]:
n = 308
a = print_adversarial_example(dataset.vec.map2words(dataset.test_data.X[n]), dataset.test_data.attn_hat[n], emax_adv_attn[n], latex=True)
dataset.test_data.yt_hat[n], emax_ad_y[n]
print(a[2])

In [ ]:
ent = [np.max(x) for x in emax_adv_attn]
plt.scatter(ent, emax_jds)

In [ ]:
idx_1 = set(np.where(np.array(ent) > 0.6)[0]) & set(np.where(np.array(emax_jds) > 0.4)[0])
idx_1

In [ ]:
train_dataset(dataset, 'logodds_lstm_reg')

In [ ]:
run_logodds_experiment(dataset, 'logodds_lstm_reg')

In [ ]:
run_experiments_on_latest_model(dataset, 'logodds_lstm_reg')

In [ ]:
generate_graphs_on_latest_model(dataset, 'logodds_lstm_reg')

In [ ]:
run_logodds_substitution_experiment(dataset)

In [ ]:
test_data = dataset.test_data
plt.scatter(test_data.yt_hat[:, 0], test_data.opp_yt_hat[:, 0], s=5)
plt.show()

In [ ]:
diff = np.abs(test_data.yt_hat[:, 0] - test_data.opp_yt_hat[:, 0])
np.argsort(diff)[:15]

In [ ]:
plt.hist(diff, bins=30)

In [ ]:
test_data = dataset.test_data
for k, v in test_data.logodds_combined[0].items() :
    print(dataset.vec.map2words(v))

In [ ]:
n = 4231
true_X = dataset.vec.map2words(test_data.X[n])
new_X = dataset.vec.map2words(test_data.opp_X[n])
print_attn(true_X, dataset.test_data.attn_hat[n])
print(test_data.yt_hat[n])
print_attn(new_X, dataset.test_data.opp_attn[n])
print(test_data.opp_yt_hat[n])

In [ ]:
for k in datasets_ehr :
    dataset = datasets_ehr[k]()
    generate_graphs_on_latest_model(dataset)

In [ ]:
for k in datasets_ehr :
    dataset = datasets_ehr[k]()

In [ ]:
for k in datasets_ehr :
    dataset = datasets_ehr[k]()
    train_dataset(dataset, 'logodds_lstm')

In [ ]:
for k in datasets_ehr :
    dataset = datasets_ehr[k]()
    run_evaluator_on_latest_model(dataset, 'logodds_lstm')

In [ ]:
from model.LR import LR

In [ ]:
for k in datasets_ehr :
    if k != 'pheno' : continue
    dataset = datasets_ehr[k]()
    train_lr_on_dataset(dataset)

In [ ]:
for k in datasets_ehr :
    dataset = datasets_ehr[k]()
    push_all_models(dataset, dataset.keys_to_use)

In [ ]:
for k in datasets_ehr :
    dataset = datasets_ehr[k]()
    run_logodds_experiment(dataset)

In [ ]:
list(enumerate(dataset.vec.label_headers))

In [ ]:
get_top_words(dataset, config='lstm')

In [ ]:
y = np.array(dataset.test_data.yt_hat)
idx_y = np.where(y > 0.8)[0]

In [ ]:
from collections import defaultdict
top_words_dict = defaultdict(float)
for i in idx_y :
    d = dataset.test_data.top_words_attn[i]
    for k, v in d.items() :
        top_words_dict[k] += v

In [ ]:
top_words = dict(sorted(top_words_dict.items(), key=lambda x: x[1])[-20:])

In [ ]:
top_words

In [ ]:
lr.print_all_features(n=40)

In [ ]:
top_words_attn = set(top_words.keys())
top_words_lr = set(lr.get_features(n=20))

In [ ]:
top_words_attn & top_words_lr

In [ ]:
len(top_words_attn & top_words_lr) / len(top_words_attn | top_words_lr)

In [ ]:
dataset.keys_to_use

Multi Adversarial Examples
==========================

In [ ]:
for k in datasets_ehr :
    dataset = datasets_ehr[k]()
#     generate_adversarial_examples(dataset, config='lstm')
#     generate_logodds_examples(dataset, config='lstm')
#     generate_graphs_on_latest_model(dataset, config='lstm')
    push_all_models(dataset, keys=dataset.keys_to_use)

In [ ]:
dataset.vec.label_headers

In [ ]:
for k in datasets_ehr :
    dataset = datasets_ehr[k]()
    dataset.display_stats()

In [ ]:
len(dataset.test_data.X)

In [ ]:
evaluator = run_evaluator_on_latest_model(dataset)

In [ ]:
multi_adversarial_outputs = pload(evaluator.model, 'multi_adversarial')

In [ ]:
from Trainers.PlottingBC import *
test_data = dataset.test_data
emax_jds, emax_adv_attn, emax_ad_y = plot_multi_adversarial(test_data.X, test_data.yt_hat, 
                                                            test_data.attn_hat, multi_adversarial_outputs, dirname=".")

In [ ]:
print_adversarial_examples(dataset, test_data.X, test_data.yt_hat, test_data.attn_hat, 
                           emax_jds, emax_adv_attn, emax_ad_y, by_class=None, dirname='.')

In [ ]:
import numpy as np
a = np.random.randn(100, 10) + 100

In [ ]:
a.mean(0).min(), a.min(0).mean()